In [1]:
import sys
import getpass
import geopandas as gpd
from arcgis import GIS
from arcgis import features

user = getpass.getuser()
sys.dont_write_bytecode = True

# sys.path.insert(0, '/Users/{}/Box/Utility Code'.format(user))

# for macOS Monterey
sys.path.insert(0, '/Users/{}/Library/CloudStorage/Box-Box/Utility Code'.format(user))

from utils_io import *

/Users/jcroff/anaconda3/envs/geo_env/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
gis = GIS(
    url='https://mtc.maps.arcgis.com/home/', 
    username='content_MTC'
)

Enter password: ········


In [43]:
work_dir = os.path.join(
    '/Users',
    user,
    'Library',
    'CloudStorage',
    'Box-Box',
    'DataViz Projects',
    'Adhoc-Spatial-Analysis',
    'Mobility Hubs Overlay Analysis'
)

## Mobility Hubs Overlay Analysis

**Analysis requirements:** 
1. mobility hub locations that are both in an EPC and transit oriented community area and are on the existing MTC bike network
2. mobility hub locations that are in a transit rich community area and on the existing MTC bike network

**Box Directory**
[Mobility Hubs Overlay Analysis Box](https://mtcdrive.box.com/s/xbl4frhez31wkucrba9ml9x1zen5wr0c)

**Data Sources**
1. [Mobility Hubs](https://mtc.maps.arcgis.com/home/item.html?id=6f3f6556d3c14d3d94ecef85270f52d4)
2. [Equity Priority Communities](https://mtc.maps.arcgis.com/home/item.html?id=28a03a46fe9c4df0a29746d6f8c633c8)
3. [Transit-Oriented Communities](https://mtc.maps.arcgis.com/home/item.html?id=3df9742c87a849a8a09342bc58da2dd6)
5. [Existing MTC Bike Network](https://mtc.maps.arcgis.com/home/item.html?id=df9ff2da527140e1ab36dd6c7d5ea0e6)

## Pull data sources

In [44]:
mh_gdf = (gpd.read_file(
    'https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/mtc_mobility_hubs_2020/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson')
          .to_crs('EPSG:26910')
         )

In [45]:
epc_gdf = (gpd.read_file(
    'https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/communities_of_concern_2020_acs2018/FeatureServer/0/query?where=1%3D1&outFields=geoid,epc_2050,epc_class&outSR=4326&f=json')
           .to_crs('EPSG:26910')
          )

In [46]:
bn_gdf = (gpd.read_file(
    os.path.join(work_dir, 'data','mtc_bike_network.geojson'))
          .to_crs('EPSG:26910')
         )

In [47]:
# pull toc item from AGOL
toc = gis.content.get(itemid='3df9742c87a849a8a09342bc58da2dd6')

In [48]:
# select layer from item
toc_layer = toc.layers[0]

In [49]:
# query layer
toc_qry = toc_layer.query(out_sr={"wkid" : 4326})

In [50]:
# export to geojson
toc_geojson = toc_qry.to_geojson

In [51]:
# create geodataframe from object and set crs as EPSG:26910 
# the default geom from AGOL was incorrectly set as EPSG:4326
toc_gdf = (gpd.read_file(
    toc_geojson)
           .to_crs('EPSG:26910')
          )

In [52]:
gg_gdf = (gpd.read_file(
    'https://services3.arcgis.com/i2dkYWmb4wHvYPda/arcgis/rest/services/pba2050_growth_geographies_2020/FeatureServer/0/query?where=1%3D1&outFields=designatio&outSR=4326&f=json')
          .to_crs('EPSG:26910')
         )

## Perform spatial joins

In [53]:
# filter epc
epc_gdf = epc_gdf[epc_gdf['epc_2050'] == 1]

In [54]:
# spatial join mobility hubs to equity priority communities
mh_epc = gpd.sjoin(
    mh_gdf, 
    epc_gdf, 
    how='left'
)

In [55]:
# create epc flag column
mh_epc['epc_pba2050'] = np.where(mh_epc['index_right'].isnull(),0,1)

In [56]:
# spatially join mobility hubs to bike network
mh_bn = gpd.sjoin_nearest(
    mh_gdf,
    bn_gdf,
    how='left',
    max_distance=15.24
)

In [57]:
# create bike network flag column
mh_bn['existing_bike_nw'] = np.where(mh_bn['index_right'].isnull(),0,1)

In [58]:
# spatially join mobility hubs to bike network within 1/4 mile
mh_1_4_mi_bn = gpd.sjoin_nearest(
    mh_gdf,
    bn_gdf,
    how='left',
    max_distance=402.336
)

In [59]:
# create 1/4 mile bike network flag column
mh_1_4_mi_bn['1_4_mi_existing_bike_nw'] = np.where(mh_1_4_mi_bn['index_right'].isnull(),0,1)

In [60]:
# spatially join mobility hubs to transit rich geographies
mh_toc = gpd.sjoin(
    mh_gdf,
    toc_gdf,
    how='left'
)

In [61]:
mh_toc['toc'] = np.where(mh_toc['index_right'].isnull(),0,1)

## Flag overlay areas within original df

In [62]:
def create_flag_column(sj_gdf, flag_col_name, original_gdf, original_gdf_id_col):
    if sj_gdf.shape[0] != original_gdf.shape[0]:
        original_gdf[flag_col_name] = original_gdf[original_gdf_id_col].map(
            sj_gdf.groupby(original_gdf_id_col)[flag_col_name].first())
    else:
        original_gdf[flag_col_name] = original_gdf[original_gdf_id_col].map(
            sj_gdf.set_index(original_gdf_id_col)[flag_col_name])

In [63]:
# create epc flag in original gdf
create_flag_column(
    sj_gdf=mh_epc,
    flag_col_name='epc_pba2050',
    original_gdf=mh_gdf,
    original_gdf_id_col='id_x'
)

In [64]:
# create bike network flag in original gdf
create_flag_column(
    sj_gdf=mh_bn,
    flag_col_name='existing_bike_nw',
    original_gdf=mh_gdf,
    original_gdf_id_col='id_x'
)

In [65]:
# create 1/4 bike network flag in original gdf
create_flag_column(
    sj_gdf=mh_1_4_mi_bn,
    flag_col_name='1_4_mi_existing_bike_nw',
    original_gdf=mh_gdf,
    original_gdf_id_col='id_x'
)

In [66]:
# create transit rich flag in original gdf
create_flag_column(
    sj_gdf=mh_toc,
    flag_col_name='toc',
    original_gdf=mh_gdf,
    original_gdf_id_col='id_x'
)

In [67]:
rename_dict = {
    'csha_flag':'careshare',
    'bknt_flag':'former_bike_nw',
    'bwsa_flag':'bay_wheels'
}
mh_gdf = mh_gdf.rename(columns=rename_dict)

In [68]:
final_cols = [
    'id_x',
    'hub_type',
    'system_typ',
    'hct_stn_nm',
    'raw_name',
    'careshare',
    'former_bike_nw',
    'bay_wheels',
    'epc_pba2050',
    'existing_bike_nw',
    '1_4_mi_existing_bike_nw',
    'toc',
    'geometry'
]
mh_gdf[final_cols].to_csv(os.path.join(work_dir,'data','mobility_hubs_overlay_analysis.csv'))

In [69]:
# #export bike network w/ missing geoms
# bn_gdf[bn_gdf['geometry'].isnull()].to_csv('data/existing_bike_nw_missing_geometry.csv',index=False)